## Target

Assume we have trained $K$ models and their predicted probabilities/distributions on the training set can be written as a matirx $A^{(k)}$ and we want to fuse them like $\sum_{k=1}^K \alpha_k A^{(k)}$ with constraint $\sum_{k=1}^K \alpha_k=1$

## Method

The method is to maximize loglikelihood
$$\frac{1}{N}\sum_{i=1}^N (\log\sum_{k=1}^K \alpha_k A^{(k)})_{i, :} \cdot l_{i,:}$$
Here $l$ is the label matrix and $l_{i,j}=1$ if the label of $i$-th sample is $j$

To be exact, let $A = \sum_{k=1}^K \alpha_k A^{(k)}$, .i.e $A_{i, j} = \sum_{k=1}^K \alpha_k A^{(k)}_{i, j}$, and there are in total $L$ classes, the average loss is
$$\mathcal{L}=\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^L\log(A_{i,j})\cdot l_{i,j}=\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^L\log(\sum_{k=1}^K \alpha_k A^{(k)}_{i,j})\cdot l_{i,j}$$

Now we consider the gradient w.r.t $\alpha_i, i=1, 2, \dots, K-1$

\begin{align*}
\frac{\partial \mathcal{L}}{\partial \alpha_m} &= \frac{1}{N}\sum_{i=1}^N\sum_{j=1}^L\frac{\partial \log(\sum_{k=1}^K \alpha_k A^{(k)}_{i,j})}{\partial \alpha_m}\cdot l_{i,j}\\
&=\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^L\frac{\frac{\partial \sum_{k=1}^K \alpha_k A^{(k)}_{i,j}}{\partial \alpha_m}}{\sum_{k=1}^K \alpha_k A^{(k)}_{i,j}}\cdot l_{i,j}\\
&=\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^L\frac{\frac{\partial (\alpha_m A^{(m)}_{i, j}+\alpha_K A^{(K)}_{i,j})}{\partial \alpha_m}}{\sum_{k=1}^K \alpha_k A^{(k)}_{i,j}}\cdot l_{i,j}\\
&=\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^L\frac{A^{(m)}_{i, j} - A^{(K)}_{i, j}}{\sum_{k=1}^K \alpha_k A^{(k)}_{i,j}}\cdot l_{i,j}
\end{align*}
    

## Regularization

Now we consider some regularization : we penalize the vector $\mathbf{\alpha}$ if it's too far away from uniform distribution. We consider two kinds of regularizations :

### KL distance 

The new target to maximize would be
      
$$\mathcal{L} -\lambda (-\sum_{k=1}^K \frac{1}{K}\log(K\alpha_k)) = \mathcal{L} + \frac{\lambda}{K}\sum_{k=1}^K \log(K\alpha_k) $$

And the new gradient becomes 
$$\frac{\partial \mathcal{L}}{\partial \alpha_m} + \frac{\lambda}{K}\cdot(\frac{1}{\alpha_m}-\frac{1}{\alpha_K})$$
### $L_2$ norm
The new target to maximize would be
      
$$\mathcal{L} -\lambda \sum_{k=1}^K (\alpha_k-\frac{1}{K})^2$$

And the new gradient becomes 
$$\frac{\partial \mathcal{L}}{\partial \alpha_m} -2\lambda(\alpha_m-\frac{1}{K})-2\lambda(-1)(\alpha_K-\frac{1}{K})=\frac{\partial \mathcal{L}}{\partial \alpha_m}-2\lambda(\alpha_m - \alpha_K)$$

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from multiprocessing import Pool
from transforms import *
from sklearn.metrics import log_loss, accuracy_score

features = ['Parameter'+str(i) for i in [5, 6, 7, 8, 9, 10]]

training = pd.read_csv('first_round_training_data.csv')
code = {'Pass':1, 'Good':2, 'Excellent':3, 'Fail':0}
training['new_Quality'] = training['Quality_label'].apply(lambda x : code[x])

L = np.zeros((training.shape[0], 4))
labels = ['Fail', 'Pass', 'Good', 'Excellent']
for i in range(4):
    L[:, i] = (training['Quality_label'].values==labels[i]).astype('float')

N = 10

skf = StratifiedKFold(n_splits=N, shuffle=True, random_state=302)
indices = []
for train_index, test_index in skf.split(training[features], training[['new_Quality']]):
    indices.append([train_index, test_index])

group_columns = []
for group in range(100):
    name = 'group_%s'%group
    group_columns.append(name)
    training[name] = 0
    kfold=StratifiedKFold(n_splits=120, shuffle=True, random_state=group)
    split=kfold.split(training[features], training['new_Quality'])
    i = 0
    for train_index, valid_index in split:
        training.iloc[valid_index,-1] = i
        i+=1

pool = Pool(processes = N_split)

#names = []
#A_list = []
#for name in names:
#    A_list.append(pd.read_csv('{}.csv'.format(name)).values)

In [3]:
def gradient(A_list, L, alpha, lambd, regularization='KL'):
    '''
    Inputs:
    =======
    A_list : list of numpy arrays containing prediction probabilities
    L : true 0-1 numpy array
    alpha : length K vector as fusion weights
    lambda : value of lambda, the strength of regularization term
    regularization : string for choice of regularization. default 'KL'. Can be 'L2'. If neither, 'KL' would be use
    
    Return:
    =======
    grads : lenght K-1 vector, which is the gradient of first K-1 elements of alpha
    '''
    if not regularization in ['KL', 'L2']:
        regularization = 'KL'
    
    N = A_list[0].shape[0]
    K = len(A_list)
    A = np.zeros(A_list[0].shape)
    for k in range(K):
        A += alpha[k] * A_list[k]
    grad = np.zeros(K-1)
    for i in range(K-1):
        grad[i] = np.mean(np.sum((A_list[i] - A_list[-1])/(A+1e-6)*L, axis=1))#partial L/partial alpha_i
    for i in range(K-1):
        if regularization=='KL':
            grad[i] += lambd/K * (1/alpha[i]-1/alpha[-1])
        else:
            grad[i] -= 2*lambd*(alpha[i] - alpha[-1])
    
    return grad

In [4]:
def optimize(A_list, L, step_size, lambd=0, regularization='KL', max_iter=100000, eps=1e-7):
    K = len(A_list)
    alpha = np.ones(K)/K
    counter = 0
    log = []
    if not regularization in ['KL', 'L2']:
        regularization = 'KL'
    while counter<max_iter:
        log.append(target_function(A_list, alpha, L, lambd, regularization))
        grads = np.zeros(K-1)
        grads = gradient(A_list, L, alpha, lambd, regularization)
        if np.mean(grads**2)/np.mean(alpha[:(K-1)]**2) < eps:
            return alpha, log, counter
        alpha[:(K-1)] += step_size * grads
        alpha[-1] = 1 - np.sum(alpha[:(K-1)])
        counter += 1
    return alpha, log, counter

In [5]:
def target_function(A_list, alpha, L, lambd, regularization):
    K = len(A_list)
    A = np.zeros(A_list[0].shape)
    for i in range(K):
        A += A_list[i] * alpha[i]
    likelihood = np.mean(np.sum(np.log(A)*L, axis=1))
    if regularization == 'KL':
        reg = lambd/K * np.sum(np.log(K*alpha))
    else:
        reg = -lambd * np.sum((lambd - 1/K)**2)
    return result, result + reg

In [1]:
import numpy as np
a = np.random.standard_normal((10, 4))
a

array([[-2.10229891, -1.0208783 , -2.52738293, -0.01370089],
       [-0.03743895, -0.6110007 , -0.48760411, -0.93041988],
       [-0.93339405, -0.75483713, -1.12733242, -0.75964912],
       [ 2.97831053, -2.38009137,  1.66008827, -0.82310972],
       [-0.38318593, -0.07503572, -1.09066392,  0.29695162],
       [-0.98612851, -0.48129729, -2.3183331 , -0.53266503],
       [ 1.15484121, -1.39424975,  0.32953798, -1.14835492],
       [ 0.31390838, -1.56448825,  0.12998619,  0.33470588],
       [ 0.29359795,  0.96648361, -0.70890427,  0.73794366],
       [ 0.35818055,  1.67546872, -1.14301274, -1.47683964]])

In [3]:
np.argmax(a, 1)

array([3, 0, 1, 0, 3, 1, 0, 3, 1, 1])